# _Experiments: March 24, 2020_

**TL;DR** --> Primary focus will be on cleaning data into an appropriate format for EDA.

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
# import libraries
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import random
import os
from tqdm import tqdm
tqdm.pandas()

# Matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

/opt/conda/envs/fastai/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## _Load Data_

In [4]:
os.getcwd()

'/notebooks/CovidDisinfo-Detect/experiments'

In [5]:
origpath = "/notebooks/CovidDisinfo-Detect/experiments"
datapath = "/notebooks/CovidDisinfo-Detect/data/raw"
filename = "coronavirus_0317_0323.json"

In [6]:
def load_data(origpath, datapath, filename):
    """
    Given path to a specific data directory, loads in data from given filename
    """
    # change directory to where data is located
    os.chdir(datapath)
    # load in data with given filename
    df = pd.read_json(filename, lines=True)
    # change directory back to original
    os.chdir(origpath)
    # return dataframe
    return df

In [7]:
%%time
df = load_data(origpath, datapath, filename)

CPU times: user 12 s, sys: 3.3 s, total: 15.3 s
Wall time: 15.3 s


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300011 entries, 0 to 300010
Data columns (total 34 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   id               300011 non-null  int64         
 1   conversation_id  300011 non-null  int64         
 2   created_at       300011 non-null  datetime64[ns]
 3   date             300011 non-null  datetime64[ns]
 4   time             300011 non-null  object        
 5   timezone         300011 non-null  object        
 6   user_id          300011 non-null  int64         
 7   username         300011 non-null  object        
 8   name             300011 non-null  object        
 9   place            300011 non-null  object        
 10  tweet            300011 non-null  object        
 11  mentions         300011 non-null  object        
 12  urls             300011 non-null  object        
 13  photos           300011 non-null  object        
 14  replies_count    300

In [9]:
df[:3]

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1241877316734717953,1241877316734717952,2020-03-22 23:59:59,2020-03-22,16:59:59,PDT,4128946034,sanjacintoclan,SanJacintoClan,,"Trump, at White House briefing, outlines broad...",[],[https://www.foxnews.com/politics/trump-corona...,[],0,0,1,[],[],https://twitter.com/SanJacintoClan/status/1241...,False,,0,,,,,,,"[{'user_id': '4128946034', 'username': 'SanJac...",,,,
1,1241877316378202115,1241873015908040704,2020-03-22 23:59:59,2020-03-22,16:59:59,PDT,393073343,powervoiceover,Power VO,,You're first tested against a panel of common ...,[shellbell_xo],[],[],0,0,0,[],[],https://twitter.com/PowerVoiceOver/status/1241...,False,,0,,,,,,,"[{'user_id': '393073343', 'username': 'PowerVo...",,,,
2,1241877315648401408,1241877315648401408,2020-03-22 23:59:59,2020-03-22,16:59:59,PDT,858501829,harrietnix,"Harriet Nix, The Egyptian Queen",,BREAKING: Rand Paul Has Tested Positive For Th...,[],[https://trendingpolitics.com/breaking-rand-pa...,[],0,1,1,[],[],https://twitter.com/HarrietNix/status/12418773...,False,,0,,,,,,,"[{'user_id': '858501829', 'username': 'Harriet...",,,,


## _Cleaning Data_

In [10]:
def list2string(df, column):
    """
    Given a dataframe and column name, converts observations from list types to string types
    """
    return df[column].apply(lambda x: ",".join([component for component in x]))

In [11]:
def datetime_edit(df):
    """
    Given a dataframe, localizes to PDT and then converts to UTC timezone
    """
    # sort observations be created_at column and then set as index
    df = df.sort_values("created_at", ascending=False).set_index("created_at").copy()
    # localize to PDT
    df = df.tz_localize("America/Los_Angeles", nonexistent="shift_backward", ambiguous=True)
    # convert to UTC timezone
    df = df.tz_convert("utc")
    return df

In [12]:
def replyto_edit(df):
    """
    Given a dataframe, extracts various information from reply_to column
    """
    # extract user_ids from reply_to column
    df["reply_to_userids"] = df["reply_to"].apply(lambda x: ",".join([str(obs["user_id"]) for obs in x]))
    # extract usernames
    df["reply_to_usernames"] = df["reply_to"].apply(lambda x: ",".join([str(obs["username"]) for obs in x]))
    # drop reply_to column
    df = df.drop(columns=["reply_to"])
    return df

In [13]:
def dataframe_clean(df):
    """
    Given a dataframe, cleans it into appropriate format for EDA
    """
    # update created_at column to reflect UTC timezone & set as index
    df = datetime_edit(df)
    # drop initial columns
    df = df.drop(columns = [
        "date", "time", "timezone", "place", "cashtags", "near", "geo", "source", "user_rt_id", "user_rt", 
        "retweet_id", "retweet_date", "translate", "trans_src", "trans_dest"
    ])
    # clean up mentions column
    print("Cleaning mentions column...")
    df["mentions"] = list2string(df, "mentions")
    # clean up urls column
    print("Cleaning urls column...")
    df["urls"] = list2string(df, "urls")
    # clean up photos column
    print("Cleaning photos column...")
    df["photos"] = list2string(df, "photos")
    # clean up hashtags column
    print("Cleaning hashtags column...")
    df["hashtags"] = list2string(df, "hashtags")
    # extract information from reply_to column into two new columns
    print("Extracting information from reply_to column...")
    df = replyto_edit(df)
    # replace all blanks with "none"
    print("Replacing all blank observations with none...")
    df = df.replace({"": "none"})
    # return updated dataframe
    print("Done!")
    return df

In [14]:
%%time

df_clean = dataframe_clean(df)

Cleaning mentions column...
Cleaning urls column...
Cleaning photos column...
Cleaning hashtags column...
Extracting information from reply_to column...
Replacing all blank observations with none...
Done!
CPU times: user 5.08 s, sys: 459 ms, total: 5.54 s
Wall time: 5.54 s


In [15]:
df_clean[:5]

,id,conversation_id,user_id,username,name,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,link,retweet,quote_url,video,reply_to_userids,reply_to_usernames
created_at,,,,,,,,,,,,,,,,,,,
2020-03-23 06:59:59+00:00,1241877316734717953,1241877316734717952,4128946034,sanjacintoclan,SanJacintoClan,"Trump, at White House briefing, outlines broad...",none,https://www.foxnews.com/politics/trump-coronav...,none,0,0,1,none,https://twitter.com/SanJacintoClan/status/1241...,False,none,0,4128946034,SanJacintoClan
2020-03-23 06:59:59+00:00,1241877315648401408,1241877315648401408,858501829,harrietnix,"Harriet Nix, The Egyptian Queen",BREAKING: Rand Paul Has Tested Positive For Th...,none,https://trendingpolitics.com/breaking-rand-pau...,none,0,1,1,none,https://twitter.com/HarrietNix/status/12418773...,False,none,0,858501829,HarrietNix
2020-03-23 06:59:59+00:00,1241877315623112704,1241877315623112704,614853,vajra,E Brown,Spanish opera singer Placido Domingo tests pos...,none,https://reut.rs/2WwhA3H,none,0,0,0,none,https://twitter.com/vajra/status/1241877315623...,False,none,0,614853,vajra
2020-03-23 06:59:59+00:00,1241877315447148544,1241877315447148544,374822255,skift,Skift,"This week, travel startups @bimbleapp and @Pru...","bimbleapp,pruvoweb",https://bit.ly/3949hyr,none,0,0,2,none,https://twitter.com/skift/status/1241877315447...,False,none,0,"374822255,842419111162482689,717347878344257536","skift,bimbleapp,PruvoWeb"
2020-03-23 06:59:59+00:00,1241877312871845891,1241877312871845888,489756858,gpbgeorge,GEORGE(Dancing&more),Coronavirus: IOC considers postponing 2020 Tok...,none,https://twitter.com/i/events/1219057585707315201,none,0,0,1,#coronavirus,https://twitter.com/GPBGeorge/status/124187731...,False,none,0,489756858,GPBGeorge


In [16]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 300011 entries, 2020-03-23 06:59:59+00:00 to 2020-03-22 23:57:53+00:00
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   id                  300011 non-null  int64 
 1   conversation_id     300011 non-null  int64 
 2   user_id             300011 non-null  int64 
 3   username            300011 non-null  object
 4   name                300011 non-null  object
 5   tweet               300011 non-null  object
 6   mentions            300011 non-null  object
 7   urls                300011 non-null  object
 8   photos              300011 non-null  object
 9   replies_count       300011 non-null  int64 
 10  retweets_count      300011 non-null  int64 
 11  likes_count         300011 non-null  int64 
 12  hashtags            300011 non-null  object
 13  link                300011 non-null  object
 14  retweet             300011 non-null  bool  
 15  quote

In [17]:
# see how many unique values are in each column
for col in list(df_clean.columns):
    print(f"Number of unique values in {col}: {len(df_clean[col].unique())}")

Number of unique values in id: 300011
Number of unique values in conversation_id: 274839
Number of unique values in user_id: 200495
Number of unique values in username: 200495
Number of unique values in name: 186386
Number of unique values in tweet: 271553
Number of unique values in mentions: 48670
Number of unique values in urls: 93140
Number of unique values in photos: 38203
Number of unique values in replies_count: 440
Number of unique values in retweets_count: 921
Number of unique values in likes_count: 1480
Number of unique values in hashtags: 54007
Number of unique values in link: 300011
Number of unique values in retweet: 1
Number of unique values in quote_url: 13926
Number of unique values in video: 2
Number of unique values in reply_to_userids: 229181
Number of unique values in reply_to_usernames: 229184


In [18]:
from datetime import datetime

datetime.now().strftime("%Y%m%d")

'20200324'

In [19]:
origpath = "/notebooks/CovidDisinfo-Detect/experiments"
datapath = "/notebooks/CovidDisinfo-Detect/data/interim"
#filename = "coronavirus_0317_0323.json"

In [20]:
def save_dataframe(df, origpath, datapath):
    """
    Given a dataframe, saves it as a pickle file and stores in correct data folder.
    """
    # change directory to where data is located
    os.chdir(datapath)
    # get current date from filename
    date = datetime.now().strftime("%Y%m%d")
    # load in data with given filename
    df.to_pickle(f"coronavirus_{date}.pkl")
    # change directory back to original
    os.chdir(origpath)
    print("Pickle file saved.")

In [21]:
save_dataframe(df_clean, origpath, datapath)

Pickle file saved.
